# Modelling the Effect of Stimulated Emission Depletion on Single Photon Emission
The quantisation of photoexcitation results in the emission of a single photon. There are various different types of Single Photon Emitters (SPEs), however we are most interested in colour centres in hexagonal Boron Nitride (hBN) due to their applicability in quantum information processing applications. Therefore, in this code values for rate constants will be taken from measured values for hBN colour centre defect, however by changing these values this code could be implimented for other SPEs.
Stimulated Emission Depletion (STED) is the process of using a photon to promote decay back to the ground state through a phonon-assisted pathway. Here we attempt to model the effect STED will have on the maximum rate of single photon emission from an SPE. It is hoped that STED can be used to increase the yeild of single photons, which would help improve the applications it is used for.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import math
import random as rnd
from scipy.optimize import curve_fit
import pandas as pd

In [3]:
def stimulatedemissionequations(w, t, p):
    """Defines the coupled first order differential equations for a 3 level system. G represents the Ground state, E 
    the excited state, and M a ground state at higher vibrational energy through which phonon assisted relaxation can 
    occur. w is a length 3 array representing the occupation of the different energy levels, t is time, and p is
    an array representing the rate constants describing the rate of change between the different energy levels."""
    G, E, M = w
    P_exc, P_STED,  Gamma1, Gamma2, Gamma3 = p
    #P_exc is the rate at which electrons are pumped from the ground to the excited state by the excitation laser
    #P_STED is the rate at which electrons are pumped from the excited state to the higher vibrational state by the STED pulse
    #Gamma1 is the rate at which electrons relax from the excited to the ground state
    #Gamma2 is the rate at which electrons relax from the excited to the higher vibrational state
    #Gamma3 is the rate at which electrons relax from the higher vibrational to the ground state
    f = [(-P_exc*G + Gamma1*E + Gamma3*M),
        (P_exc*G - (Gamma1+Gamma2+P_STED)*E),
        ((Gamma2+P_STED)*E - Gamma3*M)]
    return f

In [8]:
def StimulatedEmission(number_of_pulses, initial_occupation, pump_rate, pulse_time, no_timesteps, Gamma1,
                       Gamma2, Gamma3, stimulation_pump_rate, stimulation_pulse_time, stimulation_delay,
                      Plot_Graph=False, Show_Ground=False, Show_Excited=True, Show_Intermediate=False):
    """Calculates the evolution of the populations of the ground, excited and higher vibrational energy levels.
    stimulation_delay is the amount of time between the excitation pulse being turned off and the STED pulse being
    applied. no_timesteps is the number of data points in the calculation:a higher number will have greater accuracy
    but will be more computationally expensive. The optional argument Plot_Graph can be used to return a graph showing 
    the population of the different states. However, when running this function mulitple times it is recommended to 
    keep this off to prevent lots of graphs being produced."""
    wsol = [[initial_occupation]]
    G_plot = []
    E_plot = []
    M_plot = []
    t_total = 0
    #Calculating the total time that will be simulated, rounded up to the nearest whole integer.
    total_time = math.ceil(number_of_pulses*(pulse_time+abs(stimulation_delay)+stimulation_pulse_time))
    #Defining the errors for the numerical differential equation solver. These are just standard values.
    abserr = 1.0e-8
    relerr = 1.0e-6
    rnd_nums = []
    
    #Producing an array of the values of time that will be used in the simulation. The if statement checks that the 
    #pulse time, stimulation pulse delay and stimulation pulse time will all be an integer number of timesteps. If not,
    #the number of timesteps is increased so that it is now a multiple of the total time. This will fix the problem
        
    if ((pulse_time/total_time)*no_timesteps).is_integer() == True and ((stimulation_delay/total_time)*no_timesteps).is_integer() == True and ((stimulation_pulse_time/total_time)*no_timesteps).is_integer() == True:
        t = np.linspace(0, total_time, no_timesteps)
        
    else:
        no_timesteps = no_timesteps + total_time - (no_timesteps % total_time)        
        t = np.linspace(0, total_time, no_timesteps)
        
    #Repeating over the total number of pulses

    for j in range(number_of_pulses):

        t_subtotal = 0
        
        #The first step is when the excitation laser is turned on. Here, a random number generator is used. This is
        #becuase if the electron is already in the excited state, it can't be excited again. Therefore, if the random
        #number generator is less that the probability of the electron being in the excited state, it won't be 
        #re-excited. This might need rethinking if extending this model to no longer be looking at a single defect
        #because some electorns will be excited and some not, which won't just lead to a simple continuation of the 
        #exponential decay of the probability of being in the excited state (would return to the total population 
        #being in the excited state so in this case might need to remove the random number generator)
        
        random_number = rnd.random()
        rnd_nums.append(random_number)
        
        if random_number >= wsol[0+3*j][-1][1]:

            t_2 = []
            #working out which times will be used for this part of the calculation
            for i in range(int((pulse_time/total_time)*no_timesteps)):
                t_2.append(t[i+t_total+t_subtotal])
            #setting the rate constants
            q = [pump_rate,0,Gamma1,Gamma2,Gamma3]
            #Numerically solving the differential equation
            value = odeint(stimulatedemissionequations, wsol[0+3*j][-1], t_2, args=(q,), atol=abserr, rtol=relerr)
            wsol.append(value)
            #Appending the probability of each state being populated by the electron into an array
            for i in range(int((pulse_time/total_time)*no_timesteps)):
                G_plot.append(wsol[1+3*j][i][0])
                E_plot.append(wsol[1+3*j][i][1])
                M_plot.append(wsol[1+3*j][i][2])
            t_subtotal += len(t_2)
        
        else:
            
            t_2 = []
            for i in range(int((pulse_time/total_time)*no_timesteps)):
                t_2.append(t[i+t_total+t_subtotal])
            #In this case the electron isn't being re-excited so the pump rate is 0
            q = [0,0,Gamma1,Gamma2,Gamma3]
            value = odeint(stimulatedemissionequations, wsol[0+3*j][-1], t_2, args=(q,), atol=abserr, rtol=relerr)
            wsol.append(value)
            for i in range(int((pulse_time/total_time)*no_timesteps)):
                G_plot.append(wsol[1+3*j][i][0])
                E_plot.append(wsol[1+3*j][i][1])
                M_plot.append(wsol[1+3*j][i][2])
            t_subtotal += len(t_2)
            
            
        #The second step is the gap between the excitation laser being turned off and the STED laser being turned on.
        #Here the population of the excited state should just follow an exponential decay becuase there is nothing 
        #pumping the electrons into the excited state.
        
        t_3 = []
        for i in range(int((stimulation_delay/total_time)*no_timesteps)):
            t_3.append(t[i]+t_total+t_subtotal)
        q = [0,0,Gamma1,Gamma2,Gamma3]
        value = odeint(stimulatedemissionequations, wsol[1+3*j][-1], t_3, args=(q,), atol=abserr, rtol=relerr)
        wsol.append(value)
        for i in range(int((stimulation_delay/total_time)*no_timesteps)):
            G_plot.append(wsol[2+3*j][i][0])
            E_plot.append(wsol[2+3*j][i][1])
            M_plot.append(wsol[2+3*j][i][2])
        t_subtotal += len(t_3)
        
        #The third step is when the the STED laser is turned on. This should cause an increase in the rate of decay of
        #decay from the excited state.
        
        t_4 = []
        for i in range(int((stimulation_pulse_time/total_time)*no_timesteps)):
            t_4.append(t[i]+t_total+t_subtotal)
        q = [0,stimulation_pump_rate,Gamma1,Gamma2,Gamma3]
        value = odeint(stimulatedemissionequations, wsol[2+3*j][-1], t_4, args=(q,), atol=abserr, rtol=relerr)
        wsol.append(value)
        for i in range(int((stimulation_pulse_time/total_time)*no_timesteps)):
            G_plot.append(wsol[3+3*j][i][0])
            E_plot.append(wsol[3+3*j][i][1])
            M_plot.append(wsol[3+3*j][i][2])
        t_subtotal += len(t_4)

        t_total += t_subtotal
        
    #If the total number of pulses has passed but the total number of timesteps iterated through is less than the total
    #then work out the remaining time as if there are no lasers being applied.
        
    if t_total < no_timesteps:

        t_2 = []
        for i in range(int((no_timesteps - t_total))):
            t_2.append(t[i+t_total])
        q = [0,0,Gamma1,Gamma2,Gamma3]
        value = odeint(stimulatedemissionequations, wsol[3*number_of_pulses][-1], t_2, args=(q,), atol=abserr, rtol=relerr)
        wsol.append(value)
        for i in range(int((no_timesteps - t_total))):
            G_plot.append(wsol[1+3*number_of_pulses][i][0])
            E_plot.append(wsol[1+3*number_of_pulses][i][1])
            M_plot.append(wsol[1+3*number_of_pulses][i][2])
            
    else:
        pass
    
    #Use Simpson's rule to try and numerically calculate the integral under the ZPL intensity
    deltat = total_time/no_timesteps
    odd = 0
    for i in range(int((t_total)/2)):   #Only want to calculate up to t_total as the decay after this is only to make the code work, and is not part of the simulation
        odd += E_plot[2*i+1]
    even = 0
    for i in range(int((t_total)/2 - 1)):
        even += E_plot[2*i+2]
    integral = deltat/3 * (E_plot[0] + 4*odd + 2* even + E_plot[t_total])*Gamma1
    
    #Use Simpson's rule to try and numerically calculate the integral under the ZPL intensity as a function of time
    integral_array = []
    deltat = total_time/no_timesteps
    integral_array_time = np.arange(0,t_total*deltat, deltat)

    #Plotting a graph showing the population of the different states over time
    if Plot_Graph == True:
        plt.figure(figsize = (20, 12))
        if Show_Ground == True:
            plt.plot(t, G_plot, label = 'Ground State', linewidth = 3)
        if Show_Excited == True:
            plt.plot(t, E_plot, label = 'Excited State', linewidth = 3)
        if Show_Intermediate == True:
            plt.plot(t, M_plot, label = 'Higher Vibrational State', linewidth = 3)
        plt.xlabel('Time / ns', fontsize = 24)
        plt.ylabel('Probability of electron occupying the Energy Level', fontsize = 24)
        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)
        plt.legend(fontsize = 24)
    
        
    return t, G_plot, E_plot, M_plot, integral, integral_array, integral_array_time

In [5]:
def STED_difference(repeats, number_of_points, max_pulses, pulse_time, stimulation_pulse_time, stimulation_delay,
                   Plot_Graph=False):
    """Calculates the difference between the average number of photons produced over time with and without the STED
    pulse. Repeats is the number of repeats of each pulse, i.e. 100. Optional argument Plot_Graph can be used to plot
    a graph showing the difference with and without STED."""
    mean_integrals = [] # array containing the data without the use of STED
    mean_integrals2 = [] # array containing the data with STED being used
    
    Multiplication_constant = max_pulses/number_of_points
    
    for j in range(number_of_points): 
        print(j)
        test2 = StimulatedEmission((repeats*(j+1)*Multiplication_constant),[1,0,0],742,pulse_time,1000000,1,1,100,0,stimulation_pulse_time,stimulation_delay)
        test3 = StimulatedEmission((repeats*(j+1)*Multiplication_constant),[1,0,0],742,pulse_time,1000000,1,1,100,369,stimulation_pulse_time,stimulation_delay)
        #To increase efficiency and to reduce the effect of anomolous data at the start of the simulation, I have
        #done one pulse of Multiplication_constant*j*repeats pulses and divided the integral by
        #repeats instead of doing Multiplication_constant*j pulses 'repeats' times and
        #taking an average
        mean_integrals.append(test2[4]/repeats)
        mean_integrals2.append(test3[4]/repeats)
    
    #Calculating the mean integral per pulse (average number of photons produced per excitation pulse)
    mean_integral_per_pulse = np.zeros(len(mean_integrals))
    mean_integral_per_pulse2 = np.zeros(len(mean_integrals2))
    for i in range(len(mean_integrals)):
        mean_integral_per_pulse[i] = mean_integrals[i] / (i+1)
        mean_integral_per_pulse2[i] = mean_integrals2[i] / (i+1)
    
    #mean_integral_per_pulse seems to remain relatively constant so just take a basic average for line of best fit
    average  = np.mean(mean_integral_per_pulse)
    #average2 = np.mean(mean_integral_per_pulse2)
    #mean_integral_per_pulse seems to follow a roughly linear increase so calculate line of best fit parameters
    popt, _ = curve_fit(optimise, np.linspace(1,50,25), mean_integral_per_pulse2)
    a, b = popt
    
    if Plot_Graph == True:
        plt.figure()
        plt.scatter(np.linspace(1,max_pulses,max_pulses), mean_integral_per_pulse, label = 'No STED', color = 'blue')
        plt.scatter(np.linspace(1,max_pulses,max_pulses), mean_integral_per_pulse2, label = 'STED', color = 'orange')
        plt.axhline(average, color = 'blue')
        plt.axhline(a, color = 'orange')
        plt.xlabel('number of pulses')
        plt.ylabel('Number of photons emitted')
        plt.title('Comparison of the average number of photons per number of pulses')
        plt.legend()
        
    
    
    return mean_integral_per_pulse, mean_integral_per_pulse2, average, a, b

In [13]:
def StimulatedEmissionSetTime(number_of_pulses, initial_occupation, pump_rate, pulse_time, no_timesteps, Gamma1, 
                              Gamma2, Gamma3, stimulation_pump_rate, stimulation_pulse_time, total_time, STED_delay,
                             Plot_Graph=False, Show_Ground=False, Show_Excited=True, Show_Intermediate=False):
    """Does the same calculation as the 'StimulatedEmission' function, except instead of specifying the delay
    between pulses, instead the total time is given, and the delay between pulses then worked out from that.
    The optional argument Plot_Graph can be used to return a graph showing the population of the different states. 
    However, when running this function mulitple times it is recommended to keep this off to prevent lots of graphs 
    being produced."""
    wsol = [[initial_occupation]]
    G_plot = []
    E_plot = []
    M_plot = []
    t_total = 0
    #Instead of working out the total time, now the time between pulses is calculated instead
    time_between_pulses = (total_time - number_of_pulses * (pulse_time + stimulation_pulse_time + STED_delay)) / number_of_pulses 
    abserr = 1.0e-8
    relerr = 1.0e-6
    rnd_nums = []
        
    if ((pulse_time/total_time)*no_timesteps).is_integer() == True and ((time_between_pulses/total_time)*no_timesteps).is_integer() == True and ((stimulation_pulse_time/total_time)*no_timesteps).is_integer() == True and ((STED_delay/total_time)*no_timesteps).is_integer() == True:
        t = np.linspace(0, int(total_time), int(no_timesteps))
    else:
        no_timesteps = int(no_timesteps + total_time - no_timesteps % total_time)
        t = np.linspace(0, total_time, no_timesteps)

    for j in range(number_of_pulses):

        t_subtotal = 0
        
        random_number = rnd.random()
        rnd_nums.append(random_number)
        
        if random_number >= wsol[0+4*j][-1][1]:

            t_2 = []
            for i in range(int((pulse_time/total_time)*no_timesteps)):
                t_2.append(t[i+t_total+t_subtotal])
            q = [pump_rate,0,Gamma1,Gamma2,Gamma3]
            value = odeint(stimulatedemissionequations, wsol[0+4*j][-1], t_2, args=(q,), atol=abserr, rtol=relerr)
            wsol.append(value)
            for i in range(int((pulse_time/total_time)*no_timesteps)):
                G_plot.append(wsol[1+4*j][i][0])
                E_plot.append(wsol[1+4*j][i][1])
                M_plot.append(wsol[1+4*j][i][2])
            t_subtotal += len(t_2)
        
        else:
            
            t_2 = []
            for i in range(int((pulse_time/total_time)*no_timesteps)):
                t_2.append(t[i+t_total+t_subtotal])
            q = [0,0,Gamma1,Gamma2,Gamma3]
            value = odeint(stimulatedemissionequations, wsol[0+4*j][-1], t_2, args=(q,), atol=abserr, rtol=relerr)
            wsol.append(value)
            for i in range(int((pulse_time/total_time)*no_timesteps)):
                G_plot.append(wsol[1+4*j][i][0])
                E_plot.append(wsol[1+4*j][i][1])
                M_plot.append(wsol[1+4*j][i][2])
            t_subtotal += len(t_2)
        
        t_3 = []
        for i in range(int((STED_delay/total_time)*no_timesteps)):
            t_3.append(t[i]+t_total+t_subtotal)
        q = [0,0,Gamma1,Gamma2,Gamma3]
        value = odeint(stimulatedemissionequations, wsol[1+4*j][-1], t_3, args=(q,), atol=abserr, rtol=relerr)
        wsol.append(value)
        for i in range(int((STED_delay/total_time)*no_timesteps)):
            G_plot.append(wsol[2+4*j][i][0])
            E_plot.append(wsol[2+4*j][i][1])
            M_plot.append(wsol[2+4*j][i][2])
        t_subtotal += len(t_3)
        
        t_4 = []
        for i in range(int((stimulation_pulse_time/total_time)*no_timesteps)):
            t_4.append(t[i]+t_total+t_subtotal)
        q = [0,stimulation_pump_rate,Gamma1,Gamma2,Gamma3]
        value = odeint(stimulatedemissionequations, wsol[2+4*j][-1], t_4, args=(q,), atol=abserr, rtol=relerr)
        wsol.append(value)
        for i in range(int((stimulation_pulse_time/total_time)*no_timesteps)):
            G_plot.append(wsol[3+4*j][i][0])
            E_plot.append(wsol[3+4*j][i][1])
            M_plot.append(wsol[3+4*j][i][2])
        t_subtotal += len(t_4)
        
        t_5 = []
        for i in range(int((time_between_pulses/total_time)*no_timesteps)):
            t_5.append(t[i]+t_total+t_subtotal)
        q = [0,0,Gamma1,Gamma2,Gamma3]
        value = odeint(stimulatedemissionequations, wsol[3+4*j][-1], t_5, args=(q,), atol=abserr, rtol=relerr)
        wsol.append(value)
        for i in range(int((time_between_pulses/total_time)*no_timesteps)):
            G_plot.append(wsol[4+4*j][i][0])
            E_plot.append(wsol[4+4*j][i][1])
            M_plot.append(wsol[4+4*j][i][2])
        t_subtotal += len(t_5)

        t_total += t_subtotal
        
    if t_total < no_timesteps:

        t_2 = []
        for i in range(int((no_timesteps - t_total))):
            t_2.append(t[i+t_total])
        q = [0,0,Gamma1,Gamma2,Gamma3]
        value = odeint(stimulatedemissionequations, wsol[4*number_of_pulses][-1], t_2, args=(q,), atol=abserr, rtol=relerr)
        wsol.append(value)
        for i in range(int((no_timesteps - t_total))):
            G_plot.append(wsol[1+4*number_of_pulses][i][0])
            E_plot.append(wsol[1+4*number_of_pulses][i][1])
            M_plot.append(wsol[1+4*number_of_pulses][i][2])
            
    else:
        pass
    
    #Use Simpson's rule to try and numerically calculate the integral under the ZPL intensity
    deltat = total_time/no_timesteps
    odd = 0
    for i in range(int((t_total)/2)):
        odd += E_plot[2*i+1]
    even = 0
    for i in range(int((t_total)/2 - 1)):
        even += E_plot[2*i+2]
#     integral = deltat/3 * (E_plot[0] + 4*odd + 2* even + E_plot[t_total])*Gamma1
    integral = deltat/3 * (E_plot[0] + 4*odd + 2* even + E_plot[-1])*Gamma1

    
#     #Use Simpson's rule to try and numerically calculate the integral under the ZPL intensity
    integral_array = []
    deltat = total_time/no_timesteps
    integral_array_time = np.arange(0,t_total*deltat, deltat)

    #Plotting a graph showing the population of the different states over time
    if Plot_Graph == True:
        plt.figure(figsize = (20, 12))
        if Show_Ground == True:
            plt.plot(t, G_plot, label = 'Ground State', linewidth = 3)
        if Show_Excited == True:
            plt.plot(t, E_plot, label = 'Excited State', linewidth = 3)
        if Show_Intermediate == True:
            plt.plot(t, M_plot, label = 'Higher Vibrational State', linewidth = 3)
        plt.xlabel('Time / ns', fontsize = 24)
        plt.ylabel('Probability of electron occupying the Energy Level', fontsize = 24)
        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)
        plt.legend(fontsize = 24)
        
    return t, G_plot, E_plot, M_plot, integral, integral_array, integral_array_time

In [14]:
def Average_Photons_Per_Pulse(repeats, number_of_pulses, initial_occupation, pump_rate, pulse_time, no_timesteps,
                              Gamma1, Gamma2, Gamma3, stimulation_pump_rate, stimulation_pulse_time, total_time, 
                              STED_delay, Save_as_csv=False, csv_name='Average_Photons_Per_Pulse.csv'):
    """A function that repeats the simulation a specified number of times and calculates the average integral under
    the excited state population. This gives a value proportional to the number of single photons produced. Repeats
    are needed to account for the variability due to the random number generators which simulate the quantum nature
    of the system, however more repeats greatly increase the computational time of the function. The function can 
    take a long time to run, hence the option to save the data as a csv file is included so the function doesn't
    have to be run to generate the data every time"""
    mean_integrals = []
    for j in range(number_of_pulses):
        integral = []
        print(j)
        for i in range(repeats): 
            test2 = StimulatedEmissionSetTime(1*(j+1),initial_occupation, pump_rate, pulse_time, no_timesteps,
                              Gamma1, Gamma2, Gamma3, stimulation_pump_rate, stimulation_pulse_time, total_time, 
                              STED_delay)
            integral.append(test2[4])
        mean_integrals.append(np.mean(integral))

    mean_integral_per_pulse = np.zeros(len(mean_integrals))
    for i in range(len(mean_integrals)):
        mean_integral_per_pulse[i] = mean_integrals[i] / (2*i+1)
        
    if Save_as_csv == True:
        np.savetxt(csv_name, mean_integral_per_pulse, delimiter = ',')
    
    return mean_integral_per_pulse

The data from the Average_Photons_Per_Pulse function can be used to plot a graph of average photons vs number of pulses. Doing this both with and without STED should show that past a certain critical number of pulses in a set time (critical frequency), more photons are produced by using STED. For parameters corresponding to an emitter in the visible region in 2D hBN this critical frequency is around 340 MHz